# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 21 2022 1:27PM,240100,19,ADV80003450,"AdvaGen Pharma, Ltd",Released
1,Jun 21 2022 1:26PM,240099,10,CLI00310.1,"CLINUVEL, Inc.",Released
2,Jun 21 2022 1:26PM,240099,10,CLI00311.1,"CLINUVEL, Inc.",Released
3,Jun 21 2022 1:21PM,240097,12,8174539,"SD Head USA, LLC",Released
4,Jun 21 2022 1:19PM,240096,19,DEX0007036,Dexcel Pharma Technologies Ltd. DBA WDSrx,Released
5,Jun 21 2022 1:19PM,240096,19,DEX0007037,Dexcel Pharma Technologies Ltd. DBA WDSrx,Released
6,Jun 21 2022 1:15PM,240095,19,60017734,"GCH Granules USA, Inc.",Released
7,Jun 21 2022 1:11PM,240093,12,HH-35273,Hush Hush,Released
8,Jun 21 2022 1:11PM,240093,12,HH-35274,Hush Hush,Released
9,Jun 21 2022 1:11PM,240093,12,HH-35275,Hush Hush,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
48,240095,Released,1
49,240096,Released,2
50,240097,Released,1
51,240099,Released,2
52,240100,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
240095,NaN,NaN,1.0
240096,NaN,NaN,2.0
240097,NaN,NaN,1.0
240099,NaN,NaN,2.0
240100,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
239992,0.0,0.0,1.0
240003,0.0,0.0,1.0
240004,0.0,1.0,1.0
240006,0.0,0.0,1.0
240007,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
239992,0,0,1
240003,0,0,1
240004,0,1,1
240006,0,0,1
240007,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,239992,0,0,1
1,240003,0,0,1
2,240004,0,1,1
3,240006,0,0,1
4,240007,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,239992,,,1
1,240003,,,1
2,240004,,1,1
3,240006,,,1
4,240007,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 21 2022 1:27PM,240100,19,"AdvaGen Pharma, Ltd"
1,Jun 21 2022 1:26PM,240099,10,"CLINUVEL, Inc."
3,Jun 21 2022 1:21PM,240097,12,"SD Head USA, LLC"
4,Jun 21 2022 1:19PM,240096,19,Dexcel Pharma Technologies Ltd. DBA WDSrx
6,Jun 21 2022 1:15PM,240095,19,"GCH Granules USA, Inc."
7,Jun 21 2022 1:11PM,240093,12,Hush Hush
11,Jun 21 2022 1:10PM,240094,19,"GCH Granules USA, Inc."
12,Jun 21 2022 1:02PM,240092,19,"GUSA Granules USA, Inc."
13,Jun 21 2022 12:56PM,240090,10,Emerginnova
16,Jun 21 2022 12:38PM,240089,102,"Senseonics, Incorporated"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jun 21 2022 1:27PM,240100,19,"AdvaGen Pharma, Ltd",,,1
1,Jun 21 2022 1:26PM,240099,10,"CLINUVEL, Inc.",,,2
2,Jun 21 2022 1:21PM,240097,12,"SD Head USA, LLC",,,1
3,Jun 21 2022 1:19PM,240096,19,Dexcel Pharma Technologies Ltd. DBA WDSrx,,,2
4,Jun 21 2022 1:15PM,240095,19,"GCH Granules USA, Inc.",,,1
5,Jun 21 2022 1:11PM,240093,12,Hush Hush,,,4
6,Jun 21 2022 1:10PM,240094,19,"GCH Granules USA, Inc.",,,1
7,Jun 21 2022 1:02PM,240092,19,"GUSA Granules USA, Inc.",,,1
8,Jun 21 2022 12:56PM,240090,10,Emerginnova,,,3
9,Jun 21 2022 12:38PM,240089,102,"Senseonics, Incorporated",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 21 2022 1:27PM,240100,19,"AdvaGen Pharma, Ltd",1,,
1,Jun 21 2022 1:26PM,240099,10,"CLINUVEL, Inc.",2,,
2,Jun 21 2022 1:21PM,240097,12,"SD Head USA, LLC",1,,
3,Jun 21 2022 1:19PM,240096,19,Dexcel Pharma Technologies Ltd. DBA WDSrx,2,,
4,Jun 21 2022 1:15PM,240095,19,"GCH Granules USA, Inc.",1,,
5,Jun 21 2022 1:11PM,240093,12,Hush Hush,4,,
6,Jun 21 2022 1:10PM,240094,19,"GCH Granules USA, Inc.",1,,
7,Jun 21 2022 1:02PM,240092,19,"GUSA Granules USA, Inc.",1,,
8,Jun 21 2022 12:56PM,240090,10,Emerginnova,3,,
9,Jun 21 2022 12:38PM,240089,102,"Senseonics, Incorporated",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 21 2022 1:27PM,240100,19,"AdvaGen Pharma, Ltd",1,,
1,Jun 21 2022 1:26PM,240099,10,"CLINUVEL, Inc.",2,,
2,Jun 21 2022 1:21PM,240097,12,"SD Head USA, LLC",1,,
3,Jun 21 2022 1:19PM,240096,19,Dexcel Pharma Technologies Ltd. DBA WDSrx,2,,
4,Jun 21 2022 1:15PM,240095,19,"GCH Granules USA, Inc.",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 21 2022 1:27PM,240100,19,"AdvaGen Pharma, Ltd",1.0,NaN,NaN
1,Jun 21 2022 1:26PM,240099,10,"CLINUVEL, Inc.",2.0,NaN,NaN
2,Jun 21 2022 1:21PM,240097,12,"SD Head USA, LLC",1.0,NaN,NaN
3,Jun 21 2022 1:19PM,240096,19,Dexcel Pharma Technologies Ltd. DBA WDSrx,2.0,NaN,NaN
4,Jun 21 2022 1:15PM,240095,19,"GCH Granules USA, Inc.",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 21 2022 1:27PM,240100,19,"AdvaGen Pharma, Ltd",1.0,0.0,0.0
1,Jun 21 2022 1:26PM,240099,10,"CLINUVEL, Inc.",2.0,0.0,0.0
2,Jun 21 2022 1:21PM,240097,12,"SD Head USA, LLC",1.0,0.0,0.0
3,Jun 21 2022 1:19PM,240096,19,Dexcel Pharma Technologies Ltd. DBA WDSrx,2.0,0.0,0.0
4,Jun 21 2022 1:15PM,240095,19,"GCH Granules USA, Inc.",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,4561084,150.0,46.0,0.0
102,240089,1.0,0.0,0.0
12,480190,5.0,0.0,0.0
15,1200294,39.0,55.0,0.0
16,240070,2.0,0.0,0.0
19,1920561,9.0,2.0,2.0
20,1200336,75.0,31.0,0.0
21,239992,1.0,0.0,0.0
22,240003,1.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,4561084,150.0,46.0,0.0
1,102,240089,1.0,0.0,0.0
2,12,480190,5.0,0.0,0.0
3,15,1200294,39.0,55.0,0.0
4,16,240070,2.0,0.0,0.0
5,19,1920561,9.0,2.0,2.0
6,20,1200336,75.0,31.0,0.0
7,21,239992,1.0,0.0,0.0
8,22,240003,1.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,150.0,46.0,0.0
1,102,1.0,0.0,0.0
2,12,5.0,0.0,0.0
3,15,39.0,55.0,0.0
4,16,2.0,0.0,0.0
5,19,9.0,2.0,2.0
6,20,75.0,31.0,0.0
7,21,1.0,0.0,0.0
8,22,1.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,150.0
1,102,Released,1.0
2,12,Released,5.0
3,15,Released,39.0
4,16,Released,2.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,102,12,15,16,19,20,21,22
Status,,,,,,,,,
Completed,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0
Executing,46.0,0.0,0.0,55.0,0.0,2.0,31.0,0.0,0.0
Released,150.0,1.0,5.0,39.0,2.0,9.0,75.0,1.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,102,12,15,16,19,20,21,22
0,Completed,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0
1,Executing,46.0,0.0,0.0,55.0,0.0,2.0,31.0,0.0,0.0
2,Released,150.0,1.0,5.0,39.0,2.0,9.0,75.0,1.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,102,12,15,16,19,20,21,22
0,Completed,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0
1,Executing,46.0,0.0,0.0,55.0,0.0,2.0,31.0,0.0,0.0
2,Released,150.0,1.0,5.0,39.0,2.0,9.0,75.0,1.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()